#### Copyright 2019 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recurrent Neural Networks (RNNs)

Recurrent Neural Networks (RNNs) are an interesting application of deep learning that allow models to predict the future. While regression models attempt to fit an equation to existing data and extend the predictive power of the equation into the future, RNNs instead fit a model and use sequences of time series data to make step-by-step predictions about the next most likely output of the model.

In this colab, we will generate some artificial stock price data and then create a recurrent neural network that can predict the price of that stock into the future.

## Generating the Dataset 

Getting familiar with models is often easier when we use an artificial dataset. The data is cleaner and we know the equation that is used to create the data. This means there is definitely some pattern for our model, and we know what it is.

Let's start by importing NumPy and setting a random seed for reproducibility. Remember that you probably shouldn't set a random seed in your real code. In computers random numbers aren't really random and setting the seed allows us to use the same "randomness" for the sake of illustration.

In [0]:
import numpy as np

np.random.seed(1979)

We will create stock data for a fictional company. Let's create 10 years of stock data for a company that had and [initial public offering](https://en.wikipedia.org/wiki/Initial_public_offering) (IPO) of $5.

Let's first create an array containing the price at the end of the day for our fictional stock for 10 years. Let's assume every day is a trading day so that we don't have to contend with weekends. And let's assume that every year has 365 days, for simplicity.

For now, we'll just set the price at each day at zero.

In [0]:
days_per_year = 365
years = 10

eod_prices = np.zeros(days_per_year*years)
eod_prices.shape

We can now set the price at the IPO (day 0).

In [0]:
eod_prices[0] = 5.0

eod_prices[0:5]

Starting with the $5 IPO, let's fill the remaining prices with random data added to the IPO price with a slight positive bias.

In [0]:
bias = 0.0005

for i in range(1, len(eod_prices)):
  # Find yesterday's price.
  yesterdays_price = eod_prices[i-1]
  
  # Generate a random a percentage change on the normal curve.
  percentage_change = np.random.randn(1)[0]
  
  # The random number is a value on the standard normal distribution
  # with a mean 0 and variance 1. This will give us a nice range of
  # positive and negative values, but we need to divide by 100 to scale
  # them down to reasonable percentages for daily stock price changes.
  percentage_change /= 100

  # And finally we give the change just a little bit of positive bias
  # so that we get a nice growth curve.
  percentage_change += bias
  
  # Calculate the new price.
  todays_price = yesterdays_price + yesterdays_price * percentage_change
  
  # Store the price.
  eod_prices[i] = todays_price

We'll now use matplotlib to plot the values and make sure that it looks like a reasonable stock chart.

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(len(eod_prices))), eod_prices)
plt.show()

Nice! That actually looks like a stock chart, for a well-performing company. We'll work from this dataset to predict stock prices for this company.

## Configuring the Model

We now have a dataset of stock prices that we can use to train our model. Next, we'll build and train a model that can be used to make predictions about stock prices.

There are four primary factors that we need to consider at this point:

1. How many input features do we have?
1. How many output targets do we want?
1. How many time series steps do we want to feed the model?
1. How many layers of neurons compose the hidden layer of the model?

Let's address each of these factors in turn.

**How many input features do we have?**

In this case, we only have one: stock price.

We could have any number of other features, such as:
- the overall price of the market
- the price of the [S&P 500](https://en.wikipedia.org/wiki/S%26P_500_Index)

We could even include features that may be less useful (but who knows!), such as:
- the weather in Nantucket
- the phase of the moon

Feature engineering and selection is another topic that we've talked about in this course. When you are building your real-world models, you'll want to find features that have predictive power for your model. Don't restrict yourself to just one feature if you have other valuable inputs, but also realize that adding features adds computational cost.

In [0]:
n_inputs = 1

**How many output targets do we want?**

We are only interested in predicting the stock price on a given day, so we only want one prediction out of the model.

In [0]:
n_outputs = 1

**How many time series steps do we want to feed the model?**

Next we need to think about how many steps through time that we want to feed the model each time we pass it training data. This parameter is a little more nebulous than the input and output parameters that we have set so far.
 
The number of steps through time is the window that your model will see the world. If you make that window too small, the model won't be able to detect large cyclical patterns. Larger data can take longer to train on though.
 
We'll start with a value of 100. 100 days is just over an accounting quarter, which might be a big enough window to detect some patterns. Your selection for models you build will need to be tested and tuned to find an optimal value.

In [0]:
n_steps = 100

**How many layers of neurons compose the hidden layer of the model?**

Earlier we chose to have 100 steps, which is the depth of the network unrolled over time. Units refers to the width of the network at each point in time.

This is another one of those hyperparameters that will require some experimentation. In our case we will arbitrarily choose 25 as a starting value.

In [0]:
n_units = 25

Before creating the RNN we also reset the default graph. This is a necessity particular to Colab/Jupyter. In these environments you'll find yourself running the same block of code over and over. The underlying environment just needs to know that it is okay to start over.

Technically this isn't necessary the first time you run this code, but if you want to re-run it you should reset the graph.

In [0]:
import tensorflow as tf
tf.reset_default_graph()

We now need to create a placeholder to hold the sequence of inputs that will be provided to the model over time. In this case we are creating a placeholder that expects a single input over 100 time steps.

In [0]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

A placeholder for outputs also needs to be created.

In [0]:
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

The next step is to build a basic RNN cell. We tell the cell the number of units in each layer.

In [0]:
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)

We have a model that is 25 units wide, but all we are trying to do is predict a single stock price.

We need to bring that wide array of output down to a regression that we can use. `OutputProjectionWrapper` is capibable of doing just that.

In [0]:
cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)

And now we just need to wrap our RNN cell in a dynamic RNN wrapper.

In [0]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

The model will be tuned using an optimizer. In this case an `AdamOptimizer` will be used.

In [0]:
learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

The optimizer must know what metric to optimize. We will use the mean error between the predictions and actual y values at each step of the training.

In [0]:
loss = tf.reduce_mean(tf.square(outputs - y))
training_op = optimizer.minimize(loss)

## Training the Model

In order to train the model we need to feed it two equal-length sequences of data, one starting at time N and the other starting an N+1. This will provide the model a single shift in time to train on.

The `next_batch` function creates to equal-length arrays time shifted by one sample. The starting point for the arrays is randomly chosen.

In [0]:
def next_batch(batch_size, n_steps):
  t0 = np.empty((batch_size, n_steps, 1))
  t1 = np.empty((batch_size, n_steps, 1))

  for i in range(batch_size):
    start = np.random.randint(0, len(eod_prices) - n_steps - 1)
    t0[i] = np.array(eod_prices[start:start+n_steps]).reshape(n_steps, 1)
    t1[i] = np.array(eod_prices[start+1:start+n_steps+1]).reshape(n_steps, 1)

  return t0, t1

It is now time to decide on the number of training iterations that we want to perform and the size of the batch of training data that we want to feed the model.

We have 3,650 data points. In the example below we choose to feed the optimizer 50 100-point time series lists in a batch and do that 10,000 times. This will allow the model to train on 5,000,000 data points in various batches.

In [0]:
n_iterations = 10000
batch_size = 50

Time to train the model. We'll initialize global variables and start a TensorFlow session. Then we'll loop through each iteration feeding a batch of data to the model and log the loss every iteration while reporting it every 500 iterations.

*Note: This step might take a few minutes.*

In [0]:
loss_iter = []
loss_mse = []

with tf.Session() as sess:
  saver = tf.train.Saver()
  sess.run(tf.global_variables_initializer())
  for iteration in range(n_iterations):
    Xb, yb = next_batch(batch_size, n_steps)
    sess.run(training_op, feed_dict={X: Xb, y: yb})
    mse = loss.eval(feed_dict={X: Xb, y: yb})
    loss_iter.append(iteration)
    loss_mse.append(mse)
    if iteration % 500 == 0:
      print(f'Iter {iteration}: MSE: {mse}')
      
  saver.save(sess, "./stock_rnn")

print(f'Final MSE: {loss_mse[-1]}')

Since we stored the loss at every iteration, we can visualize the model learning over time.

In [0]:
plt.plot(loss_iter, loss_mse)
plt.show()

## Making Predictions

Our model is now trained and it should have done a pretty good job training to predict stock prices. But how do we make predictions using the model?

To do that we can feed the model stock prices for the past 100 days (our unrolled time series size) and see what the model thinks the price will be on the 101st day.

In [0]:
with tf.Session() as sess: 
  saver.restore(sess, "./stock_rnn")

  X_batch, _ = next_batch(1, n_steps) # get 100 data points
  y_pred = sess.run(outputs, feed_dict={X: [X_batch[0]]})
  
  today = X_batch[0][-1][0]
  tomorrow = y_pred[0][-1][0]

  print(f'Price today: {today:0.2f}, price tomorrow: {tomorrow:0.2f}')

We can plot our predictions against the actual values of our stock prices.

In [0]:
import matplotlib.pyplot as plt

predictions = []

with tf.Session() as sess: 
  saver.restore(sess, "./stock_rnn")

  for i in range(0, len(eod_prices)-n_steps, n_steps):
    y_pred = sess.run(outputs, feed_dict={
        X: [np.array(eod_prices[i:i+n_steps]).reshape(n_steps, 1)]})
    predictions.extend([x[0] for x in y_pred[0]])

plt.plot(list(range(len(eod_prices[1:len(predictions)]))),
         eod_prices[1:len(predictions)], color='green')
plt.plot(list(range(len(predictions[0:len(predictions)-1]))),
         predictions[0:len(predictions)-1], color='red')
plt.show()

The lines seem to be fairly similar. In a real market situation, we'd never be this accurate... But it's fun to dream!

# Exercises

## Exercise 1: Adjust Steps

Tuning a machine learning model can be as much of an art as it is a science. In this exercise we will adjust the number of time steps that are unrolled by the model and examine the results.

Keeping the random seeds the same between runs, adjust the step size to:

1. 10
1. 50
1. 100
1. 300
1. 500

Store the values for each step size in a five-element iterable called `error_rates`.

```
error_rates = [0.0, 1.0, 2.0, 3.0, 4.0]
```

As a starting point, the combined code for the model is below.



### Student Solution

In [0]:
import tensorflow as tf
import numpy as np

error_rates = [] # TBD

n_inputs = 1
n_outputs = 1
n_steps = 100
n_units = 25
batch_size = 50
n_iterations = 10000

tf.reset_default_graph()
np.random.seed(1979)
tf.random.set_random_seed(1979)

loss_iter = []
loss_mse = []

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)
cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

loss = tf.reduce_mean(tf.square(outputs - y))
training_op = optimizer.minimize(loss)

with tf.Session() as sess:
  saver = tf.train.Saver()
  sess.run(tf.global_variables_initializer())
  for iteration in range(n_iterations):
    Xb, yb = next_batch(batch_size, n_steps)
    sess.run(training_op, feed_dict={X: Xb, y: yb})
    mse = loss.eval(feed_dict={X: Xb, y: yb})
    loss_iter.append(iteration)
    loss_mse.append(mse)
    if iteration % 500 == 0:
      print("Iter {}: MSE: {}".format(iteration, mse))

  saver.save(sess, "./stock_rnn")

print(f'Final MSE: {loss_mse[-1]}')

### Answer Key

**Solution**

In [0]:
import tensorflow as tf
import numpy as np

error_rates = []

n_inputs = 1
n_outputs = 1
step_values = [10, 50, 100, 300, 500]
n_units = 25
batch_size = 50
n_iterations = 10000

for n_steps in step_values:
  tf.reset_default_graph()
  np.random.seed(1979)
  tf.random.set_random_seed(1979)

  loss_iter = []
  loss_mse = []

  X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
  y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

  rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)
  cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)
  outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

  learning_rate = 0.001
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

  loss = tf.reduce_mean(tf.square(outputs - y))
  training_op = optimizer.minimize(loss)

  with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    for iteration in range(n_iterations):
      Xb, yb = next_batch(batch_size, n_steps)
      sess.run(training_op, feed_dict={X: Xb, y: yb})
      mse = loss.eval(feed_dict={X: Xb, y: yb})
      loss_iter.append(iteration)
      loss_mse.append(mse)
      if iteration % 500 == 0:
        print("Iter {}: MSE: {}".format(iteration, mse))

    saver.save(sess, "./stock_rnn")

  print(f'Final MSE at step size {n_steps}: {loss_mse[-1]}')
  error_rates.append(loss_mse[-1])

print(error_rates)

**Validation**

In [0]:
import tensorflow as tf
import numpy as np

final_error_rates = []

n_inputs = 1
n_outputs = 1
step_values = [10, 50, 100, 300, 500]
n_units = 25
batch_size = 50
n_iterations = 10000

for n_steps in step_values:
  tf.reset_default_graph()
  np.random.seed(1979)
  tf.random.set_random_seed(1979)

  loss_iter = []
  loss_mse = []

  X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
  y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

  rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)
  cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)
  outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

  learning_rate = 0.001
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

  loss = tf.reduce_mean(tf.square(outputs - y))
  training_op = optimizer.minimize(loss)

  with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    for iteration in range(n_iterations):
      Xb, yb = next_batch(batch_size, n_steps)
      sess.run(training_op, feed_dict={X: Xb, y: yb})
      mse = loss.eval(feed_dict={X: Xb, y: yb})
      loss_iter.append(iteration)
      loss_mse.append(mse)
      if iteration % 500 == 0:
        print("Iter {}: MSE: {}".format(iteration, mse))

    saver.save(sess, "./stock_rnn")

  print(f'Final MSE at step size {n_steps}: {loss_mse[-1]}')
  final_error_rates.append(loss_mse[-1])

if error_rates == final_error_rates:
  print('LGTM')
else:
  print(f'Wanted error rates {final_error_rates}, got {error_rates}')

## Exercise 2: Adjust the Iterations

In this exercise we will adjust the number of iterations used by the model and examine the results.

Keeping the random seeds the same between runs, adjust the step size to:

1. 10
1. 100
1. 1000
1. 10000

Store the values for each step size in a four-element iterable called `error_rates`.

```
  error_rates = [0.0, 1.0, 2.0, 3.0]
```

As a starting point, the combined code for the model is below.



### Student Solution

In [0]:
import tensorflow as tf
import numpy as np

error_rates = [] # TBD

n_inputs = 1
n_outputs = 1
n_steps = 100
n_units = 25
batch_size = 50
n_iterations = 10000

tf.reset_default_graph()
np.random.seed(1979)
tf.random.set_random_seed(1979)

loss_iter = []
loss_mse = []

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)
cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

loss = tf.reduce_mean(tf.square(outputs - y))
training_op = optimizer.minimize(loss)

with tf.Session() as sess:
  saver = tf.train.Saver()
  sess.run(tf.global_variables_initializer())
  for iteration in range(n_iterations):
    Xb, yb = next_batch(batch_size, n_steps)
    sess.run(training_op, feed_dict={X: Xb, y: yb})
    mse = loss.eval(feed_dict={X: Xb, y: yb})
    loss_iter.append(iteration)
    loss_mse.append(mse)
    if iteration % 500 == 0:
      print("Iter {}: MSE: {}".format(iteration, mse))

  saver.save(sess, "./stock_rnn")

print(f'Final MSE: {loss_mse[-1]}')

### Answer Key

**Solution**

In [0]:
import tensorflow as tf
import numpy as np

error_rates = []

n_inputs = 1
n_outputs = 1
n_steps = 50
n_units = 25
batch_size = 50
iteration_size = [10, 100, 1000, 10000]

for n_iterations in iteration_size:
  tf.reset_default_graph()
  np.random.seed(1979)
  tf.random.set_random_seed(1979)

  loss_iter = []
  loss_mse = []

  X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
  y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

  rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)
  cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)
  outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

  learning_rate = 0.001
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

  loss = tf.reduce_mean(tf.square(outputs - y))
  training_op = optimizer.minimize(loss)

  with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    for iteration in range(n_iterations):
      Xb, yb = next_batch(batch_size, n_steps)
      sess.run(training_op, feed_dict={X: Xb, y: yb})
      mse = loss.eval(feed_dict={X: Xb, y: yb})
      loss_iter.append(iteration)
      loss_mse.append(mse)
      if iteration % 500 == 0:
        print("Iter {}: MSE: {}".format(iteration, mse))

    saver.save(sess, "./stock_rnn")

  print(f'Final MSE at step size {n_steps}: {loss_mse[-1]}')
  error_rates.append(loss_mse[-1])

print(error_rates)

**Validation**

In [0]:
import tensorflow as tf
import numpy as np

final_error_rates = []

n_inputs = 1
n_outputs = 1
n_steps = 50
n_units = 25
batch_size = 50
iteration_size = [10, 100, 1000, 10000]

for n_iterations in iteration_size:
  tf.reset_default_graph()
  np.random.seed(1979)
  tf.random.set_random_seed(1979)

  loss_iter = []
  loss_mse = []

  X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
  y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

  rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)
  cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)
  outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

  learning_rate = 0.001
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

  loss = tf.reduce_mean(tf.square(outputs - y))
  training_op = optimizer.minimize(loss)

  with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    for iteration in range(n_iterations):
      Xb, yb = next_batch(batch_size, n_steps)
      sess.run(training_op, feed_dict={X: Xb, y: yb})
      mse = loss.eval(feed_dict={X: Xb, y: yb})
      loss_iter.append(iteration)
      loss_mse.append(mse)
      if iteration % 500 == 0:
        print("Iter {}: MSE: {}".format(iteration, mse))

    saver.save(sess, "./stock_rnn")

  print(f'Final MSE at step size {n_steps}: {loss_mse[-1]}')
  final_error_rates.append(loss_mse[-1])

if error_rates == final_error_rates:
  print('LGTM')
else:
  print(f'Wanted error rates {final_error_rates}, got {error_rates}')

## Exercise 3: Adjust the Units

In this exercise we will adjust width of the layers in the model and examine the results.

Keeping the random seeds the same between runs, adjust the unit size to:

1. 10
1. 25
1. 50
1. 100
1. 200

Store the values for each step size in a five-element iterable called `error_rates`.

```
  error_rates = [0.0, 1.0, 2.0, 3.0, 4.0]
```

As a starting point, the combined code for the model is below.



### Student Solution

In [0]:
import tensorflow as tf
import numpy as np

error_rates = [] # TBD

n_inputs = 1
n_outputs = 1
n_steps = 100
n_units = 25
batch_size = 50
n_iterations = 10000

tf.reset_default_graph()
np.random.seed(1979)
tf.random.set_random_seed(1979)

loss_iter = []
loss_mse = []

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)
cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

loss = tf.reduce_mean(tf.square(outputs - y))
training_op = optimizer.minimize(loss)

with tf.Session() as sess:
  saver = tf.train.Saver()
  sess.run(tf.global_variables_initializer())
  for iteration in range(n_iterations):
    Xb, yb = next_batch(batch_size, n_steps)
    sess.run(training_op, feed_dict={X: Xb, y: yb})
    mse = loss.eval(feed_dict={X: Xb, y: yb})
    loss_iter.append(iteration)
    loss_mse.append(mse)
    if iteration % 500 == 0:
      print("Iter {}: MSE: {}".format(iteration, mse))

  saver.save(sess, "./stock_rnn")

print(f'Final MSE: {loss_mse[-1]}')

### Answer Key

**Solution**

In [0]:
import tensorflow as tf
import numpy as np

error_rates = []

n_inputs = 1
n_outputs = 1
n_steps = 50
unit_sizes = [10, 25, 50, 100, 200]
batch_size = 50
n_iterations = 10000

for n_units in unit_sizes:
  tf.reset_default_graph()
  np.random.seed(1979)
  tf.random.set_random_seed(1979)

  loss_iter = []
  loss_mse = []

  X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
  y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

  rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)
  cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)
  outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

  learning_rate = 0.001
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

  loss = tf.reduce_mean(tf.square(outputs - y))
  training_op = optimizer.minimize(loss)

  with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    for iteration in range(n_iterations):
      Xb, yb = next_batch(batch_size, n_steps)
      sess.run(training_op, feed_dict={X: Xb, y: yb})
      mse = loss.eval(feed_dict={X: Xb, y: yb})
      loss_iter.append(iteration)
      loss_mse.append(mse)
      if iteration % 500 == 0:
        print("Iter {}: MSE: {}".format(iteration, mse))

    saver.save(sess, "./stock_rnn")

  print(f'Final MSE at step size {n_steps}: {loss_mse[-1]}')
  error_rates.append(loss_mse[-1])

print(error_rates)

**Validation**

In [0]:
import tensorflow as tf
import numpy as np

final_error_rates = []

n_inputs = 1
n_outputs = 1
n_steps = 50
unit_sizes = [10, 25, 50, 100, 200]
batch_size = 50
n_iterations = 10000

for n_units in unit_sizes:
  tf.reset_default_graph()
  np.random.seed(1979)
  tf.random.set_random_seed(1979)

  loss_iter = []
  loss_mse = []

  X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
  y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

  rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_units)
  cell = tf.contrib.rnn.OutputProjectionWrapper(rnn_cell, 1)
  outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

  learning_rate = 0.001
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

  loss = tf.reduce_mean(tf.square(outputs - y))
  training_op = optimizer.minimize(loss)

  with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    for iteration in range(n_iterations):
      Xb, yb = next_batch(batch_size, n_steps)
      sess.run(training_op, feed_dict={X: Xb, y: yb})
      mse = loss.eval(feed_dict={X: Xb, y: yb})
      loss_iter.append(iteration)
      loss_mse.append(mse)
      if iteration % 500 == 0:
        print("Iter {}: MSE: {}".format(iteration, mse))

    saver.save(sess, "./stock_rnn")

  print(f'Final MSE at step size {n_steps}: {loss_mse[-1]}')
  final_error_rates.append(loss_mse[-1])

if error_rates == final_error_rates:
  print('LGTM')
else:
  print(f'Wanted error rates {final_error_rates}, got {error_rates}')

## Exercise 4: Your Own RNN

So far we have only played with artificial time series data. For this exercise you will import [actual stock data from Kaggle](https://www.kaggle.com/szrlee/stock-time-series-20050101-to-20171231) and attempt to predict future prices.

Download the [McDonald's stock data file](https://www.kaggle.com/szrlee/stock-time-series-20050101-to-20171231#MCD_2006-01-01_to_2018-01-01.csv) and create an RNN that will attempt to predict the closing price for the stock.

### Student Solution

In [0]:
# Your code goes here.

### Answer Key

**Solution**

In [0]:
# TODO(joshmcadams)